Создаём базу данных и Hive таблицу для потоковых данных

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("StreamingHiveExample") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# создаём новую базу
spark.sql("CREATE DATABASE IF NOT EXISTS streaming_test")

# создаём EXTERNAL TABLE на Parquet
spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS streaming_test.breweries_stream(
    NUM INT,
    NAME STRING,
    CITY STRING,
    STATE STRING,
    ID INT
)
STORED AS PARQUET
LOCATION 'hdfs://namenode:9000/data/openbeer/streaming_breweries'
""")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/09 15:47:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/09 15:47:07 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

Считываем паркеты

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("StreamingRead") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# читаем поток Parquet файлов
streaming_df = spark.readStream \
    .schema("NUM INT, NAME STRING, CITY STRING, STATE STRING, ID INT") \
    .parquet("hdfs://namenode:9000/data/openbeer/streaming_breweries")

# выводим в консоль (для теста)
query = streaming_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()
